In [5]:
import os
from datetime import datetime, timedelta
from google.cloud import storage
import pymysql
import pymssql
import pandas as pd


In [6]:

sql_server_name = '172.26.200.20'
sql_database_name = 'MIX'
sql_username = 'sa'
sql_password = 'sql25xE!'

In [14]:
sql_qry = f"""select a.FechaProduccion, a.Turno, a.TipoPasta, a.idProducto, a.Producto, 
a.Libras LibrasEntregadas, a.Rendimiento, a.Precio, a.costo,  isnull(pos.Nombre,'') pos ,
 a.Cantidad Solicitado, a.Entregado, (a.Cantidad - a.Entregado) Diferencia
from (
Select  
o.FechaProduccion,
e.Codigo as Turno,
t.Nombre as TipoPasta,
p.IdArticulo idProducto,
p.Nombre as Producto,
sum(isnull(((fd.Cantidad/f.Rendimiento) * od.Entregado) * mf.Multiplo,0))  as Libras,
f.Rendimiento  ,
   AVG( p.Precio ) Precio,
   avg( (h.costo)) costo
   , sum(od.entregado ) Entregado
   , sum(od.Cantidad) Cantidad
from Pro_Orden o
inner join Pro_OrdenDetalle od on o.IdOrden=od.IdOrden
inner join Pro_Formulacion f on od.IdArticulo=f.IdArticulo
inner join Pro_FormulacionDetalle fd on f.IdFormulacion=fd.IdFormulacion and fd.Gasto=0
inner join Gen_Articulo h on fd.IdArticulo=h.IdArticulo
inner join Gen_Entidad e on o.IdEntidad=e.IdEntidad
inner join Pro_TipoPasta t on f.IdTipoPasta=t.IdTipoPasta
inner join Gen_Articulo p on f.IdArticulo=p.IdArticulo
left join Gen_Multiplos mf on h.IdArticulo=mf.IdArticulo and mf.IdUnidadMedida = fd.IdUnidadMedida  

where o.IdEstado=7 and o.Debaja is null and f.Debaja is null and
   o.FechaProduccion >= '2020-12-31' 
     and h.Nombre LIKE '%harina%'
    -- o.FechaProduccion between DATEADD(DAY,-10, CONVERT(varchar,getdate(),111))  and CONVERT(varchar,getdate(),111)
group by o.FechaProduccion, e.Codigo, t.Nombre, p.Nombre,  f.Rendimiento , p.IdArticulo
) as a
    left join Pos_Conversion c on a.idProducto = c.IdArticulo
  left join Gen_Articulo pos on c.IdPos = pos.IdArticulo
order by a.Turno,a.FechaProduccion, a.Producto;
"""

In [15]:
# Establish a connection
try:
    # Create a connection string
    connection = pymssql.connect(
            server=sql_server_name,
            user=sql_username,
            password=sql_password,
            database=sql_database_name,
            as_dict=True
        ) 
    print("Connected to the database!")
except Exception as e:
    print(f"Error connecting to the database: {e}")

# Create a cursor
cursor = connection.cursor()
# Execute a SQL query
cursor.execute(sql_qry)
df = pd.DataFrame(cursor.fetchall())
cursor.close()
connection.close()

#change fechaprodduccion to datetime
#loaded_data_consumo['Fecha'] = pd.to_datetime(loaded_data_consumo['Fecha'])

Connected to the database!


In [16]:
df.head()

,FechaProduccion,Turno,TipoPasta,idProducto,Producto,LibrasEntregadas,Rendimiento,Precio,costo,pos,Solicitado,Entregado,Diferencia
0,2022-01-02,ALM1MOJADO,Normal,8733,batida,0.951728,31.52160,0.500000,3.266880,Pan Q0.50,80.00000,40.00000,40.00000
1,2022-01-02,ALM1MOJADO,Normal,4279,bollo tira,1.044103,28.73280,0.500000,3.526180,Pan Q0.50,40.00000,40.00000,0.00000
2,2022-01-02,ALM1MOJADO,Especial,4289,cachito especial,32.665505,6.88800,4.000000,3.266880,Pan Q4.00,252.00000,300.00000,-48.00000
3,2022-01-02,ALM1MOJADO,Especial,4311,conchita especial,16.332847,6.88796,4.000000,3.266880,Pan Q4.00,120.00000,150.00000,-30.00000
4,2022-01-02,ALM1MOJADO,Especial,4317,corona 14,0.888099,2.70240,10.000000,3.266880,Corona grande,4.00000,4.00000,0.00000


In [17]:
df.to_csv('dataMsSql.csv', index=False)